In [1]:
import os
import keras
import numpy as np
import tensorflow as tf
import neuralspot_edge as nse
import matplotlib.pyplot as plt

2024-08-20 21:41:22.559742: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-20 21:41:22.565644: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-20 21:41:22.573471: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-20 21:41:22.575836: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-20 21:41:22.581854: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Constants
AUG_SIZE = 64
CROP_SIZE = 64
NUM_CLASSES = 10
SHUFFLE_SIZE = 10000
BATCH_SIZE = 256
EPOCHS = 100
LR_RATE = 1e-3
LR_CYCLES = 1
VERBOSE = 1

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
y_train = y_train.squeeze()
y_test = y_test.squeeze()

In [4]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))

I0000 00:00:1724190084.312473   22579 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724190084.330959   22579 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724190084.331052   22579 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724190084.332882   22579 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [5]:
# # Plot a random image
# fig, ax = plt.subplots(figsize=(2, 2))
# ax.imshow(x_train[np.random.randint(0, x_train.shape[0])])
# _ = ax.axis('off')

In [6]:
augmenter = nse.layers.preprocessing.AugmentationPipeline(
    layers=[
        nse.layers.preprocessing.Resizing2D(
            height=AUG_SIZE,
            width=AUG_SIZE,
            name="Resizing"
        ),
        # nse.layers.preprocessing.RandomFlip2D(
        #     horizontal=True,
        #     vertical=False,
        #     name="RandomFlip"
        # ),
        # nse.layers.preprocessing.RandomCutout2D(
        #     factor=(0.01, 0.05),
        #     cutouts=2,
        #     fill_mode="constant",
        #     fill_value=0.0,
        #     name="RandomCutout"
        # ),
        # nse.layers.preprocessing.RandomCrop2D(
        #     height=CROP_SIZE,
        #     width=CROP_SIZE,
        #     name="RandomCrop",
        #     auto_vectorize=True
        # ),
        # nse.layers.preprocessing.Rescaling2D(
        #     scale=1.0 / 255.0,
        #     name="Rescaling"
        # )
    ],
)

In [7]:
y = augmenter(x_train[:1].astype(np.float32))

In [8]:
# # Plot a random image
# fig, ax = plt.subplots(figsize=(2, 2))
# ax.imshow(y[0].numpy())
# _ = ax.axis('off')


In [9]:
train_ds = train_ds.shuffle(
    buffer_size=SHUFFLE_SIZE
).batch(
    batch_size=BATCH_SIZE
).map(
    lambda data, labels: {
        "data": tf.cast(data, "float32"),
        "labels": tf.one_hot(labels, NUM_CLASSES),
    },
    num_parallel_calls=tf.data.AUTOTUNE,
).map(
    augmenter,
    num_parallel_calls=tf.data.AUTOTUNE
).map(
    lambda data: (data["data"], data["labels"]),
    num_parallel_calls=tf.data.AUTOTUNE,
).prefetch(
    tf.data.AUTOTUNE
)

In [10]:
val_ds = val_ds.shuffle(
    buffer_size=SHUFFLE_SIZE
).batch(
    batch_size=BATCH_SIZE
).map(
    lambda data, labels: {
        "data": tf.cast(data, "float32"),
        "labels": tf.one_hot(labels, NUM_CLASSES),
    },
    num_parallel_calls=tf.data.AUTOTUNE,
).map(
    augmenter,
    num_parallel_calls=tf.data.AUTOTUNE
).map(
    lambda data: (data["data"], data["labels"]),
    num_parallel_calls=tf.data.AUTOTUNE,
).cache().prefetch(
    tf.data.AUTOTUNE
)

In [11]:
inputs = keras.Input(shape=(CROP_SIZE, CROP_SIZE, 3), name='inputs')

model = nse.models.EfficientNetV2(
    x=inputs,
    params=nse.models.EfficientNetParams(
        input_filters=32,
        input_kernel_size=(3, 3),
        input_strides=(2, 2),
        blocks=[
            nse.models.MBConvParams(filters=48, depth=2, kernel_size=(3, 3), strides=(2, 2), ex_ratio=1, se_ratio=4),
            nse.models.MBConvParams(filters=64, depth=2, kernel_size=(3, 3), strides=(2, 2), ex_ratio=1, se_ratio=4),
            nse.models.MBConvParams(filters=96, depth=2, kernel_size=(3, 3), strides=(1, 1), ex_ratio=1, se_ratio=4),
            nse.models.MBConvParams(filters=128, depth=1, kernel_size=(3, 3), strides=(2, 2), ex_ratio=1, se_ratio=4)
        ],
        output_filters=0,
        include_top=True,
        dropout=0.2,
        drop_connect_rate=0.2,
        model_name="efficientnetv2"
    ),
    num_classes=NUM_CLASSES,
)

In [12]:
flops = nse.metrics.flops.get_flops(model, batch_size=1, fpath=os.devnull)
model.summary(layer_range=('inputs', model.layers[10].name))
print(f"FLOPS: {flops/1e6:.2f}M")

Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.


[08/20/24 21:41:24] WARNING  WARNING:tensorflow:From                                              ]8;id=271574;file:///workspaces/neuralspot-edge/.venv/lib/python3.12/site-packages/tensorflow/python/util/deprecation.py\deprecation.py]8;;\:]8;id=333808;file:///workspaces/neuralspot-edge/.venv/lib/python3.12/site-packages/tensorflow/python/util/deprecation.py#50\50]8;;\
                             /workspaces/neuralspot-edge/.venv/lib/python3.12/site-packages/tenso                  
                             rflow/python/ops/nn_ops.py:5256: tensor_shape_from_node_def_name                      
                             (from tensorflow.python.framework.graph_util_impl) is deprecated and                  
                             will be removed in a future version.                                                  
                             Instructions for updating:                                                            
                             This API was designed for TensorFlow v1. See                                          
                             https://www.tensorflow.org/guide/migrate for instructions on how to                   
                             migrate your code to TensorFlow v2.                                                   

Model: "EfficientNetV2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ inputs (InputLayer) │ (None, 64, 64, 3) │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem.conv (Conv2D)  │ (None, 32, 32,    │        864 │ inputs[0][0]      │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem.bn             │ (None, 32, 32,    │        128 │ stem.conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem.act            │ (None, 32, 32,    │          0 │ stem.bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1.mbconv1.dp   │ (None, 16, 16,    │        288 │ stem.act[0][0]    │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1.mbconv1.dp.… │ (None, 16, 16,    │        128 │ stage1.mbconv1.d… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1.mbconv1.dp.… │ (None, 16, 16,    │          0 │ stage1.mbconv1.d… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1.mbconv1.se.… │ (None, 1, 1, 32)  │          0 │ stage1.mbconv1.d… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1.mbconv1.se.… │ (None, 1, 1, 8)   │        264 │ stage1.mbconv1.s… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1.mbconv1.se.… │ (None, 1, 1, 8)   │          0 │ stage1.mbconv1.s… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1.mbconv1.se.… │ (None, 1, 1, 32)  │        288 │ stage1.mbconv1.s… │
│ (Conv2D)            │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 65,626 (256.35 KB)

 Trainable params: 63,578 (248.35 KB)

 Non-trainable params: 2,048 (8.00 KB)

FLOPS: 8.11M


In [16]:
t_mul = 1
first_steps = (EPOCHS * len(train_ds)) / (np.power(LR_CYCLES, t_mul) - t_mul + 1)
scheduler = keras.optimizers.schedules.CosineDecayRestarts(
    initial_learning_rate=LR_RATE,
    first_decay_steps=np.ceil(first_steps),
    t_mul=t_mul,
    m_mul=0.5,
)

optimizer = keras.optimizers.Adam(scheduler)
loss = keras.losses.CategoricalCrossentropy(from_logits=True)

metrics = [
    keras.metrics.CategoricalAccuracy(name="acc"),
    keras.metrics.F1Score(name="f1", average="weighted"),
]

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=25,
        restore_best_weights=True,
        verbose=VERBOSE
    ),
    keras.callbacks.ModelCheckpoint(
        monitor="val_loss",
        verbose=VERBOSE,
        save_best_only=True,
        filepath="model.keras"
    )
]


In [17]:
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics,
)

In [18]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=VERBOSE,
)

Epoch 1/100


196/196 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - acc: 0.7750 - f1: 0.7745 - loss: 0.6407
Epoch 1: val_loss improved from inf to 0.79992, saving model to model.keras
196/196 ━━━━━━━━━━━━━━━━━━━━ 18s 44ms/step - acc: 0.7750 - f1: 0.7745 - loss: 0.6407 - val_acc: 0.7270 - val_f1: 0.7266 - val_loss: 0.7999
Epoch 2/100
192/196 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.7805 - f1: 0.7800 - loss: 0.6183
Epoch 2: val_loss improved from 0.79992 to 0.79382, saving model to model.keras
196/196 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - acc: 0.7806 - f1: 0.7801 - loss: 0.6182 - val_acc: 0.7278 - val_f1: 0.7245 - val_loss: 0.7938
Epoch 3/100
195/196 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - acc: 0.7863 - f1: 0.7859 - loss: 0.5987
Epoch 3: val_loss did not improve from 0.79382
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - acc: 0.7863 - f1: 0.7859 - loss: 0.5987 - val_acc: 0.7234 - val_f1: 0.7234 - val_loss: 0.8143
Epoch 4/100
190/196 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - acc: 0.7908 - f1: 0.7905 - loss: 0.5915
Epoch 4: val_lo